In [1]:
import pandas as pd
from dateutil.parser import parse
from datetime import timedelta
import numpy as np

In [25]:
types = {'id': 'int32',
         'date': 'str',
         'item_nbr': 'int32',
         'store_nbr': 'int16',
         'unit_sales': 'float16',
         'onpromotion': bool,
    }
train = pd.read_csv('./data/train.csv', dtype=types)

C:\Users\Anatoliy\Anaconda2\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [27]:
train['onpromotion'].fillna(False, inplace=True)

In [28]:
train['date'].fillna('', inplace=True)

In [29]:
train['date'].dtype

dtype('O')

In [20]:
start = parse('2013-01-01')
end = parse('2017-08-15')

replace = {'date':{}}
days = 0
while start <= end:
    day_str = start.strftime('%Y-%m-%d')
    replace['date'][day_str] = str(days)
    start += timedelta(days=1)
    days += 1
replace

{'date': {'2013-01-20': '19',
  '2014-06-02': '517',
  '2016-04-25': '1210',
  '2015-10-05': '1007',
  '2016-08-26': '1333',
  '2014-01-10': '374',
  '2017-05-19': '1599',
  '2014-08-06': '582',
  '2013-09-01': '243',
  '2013-05-01': '120',
  '2013-03-04': '62',
  '2013-08-29': '240',
  '2013-06-09': '159',
  '2016-11-10': '1409',
  '2014-04-13': '467',
  '2016-12-20': '1449',
  '2015-11-28': '1061',
  '2013-10-22': '294',
  '2013-01-25': '24',
  '2017-07-25': '1666',
  '2017-01-26': '1486',
  '2015-03-02': '790',
  '2013-05-16': '135',
  '2015-04-15': '834',
  '2017-03-13': '1532',
  '2014-04-12': '466',
  '2016-10-20': '1388',
  '2016-01-27': '1121',
  '2015-10-29': '1031',
  '2013-05-14': '133',
  '2013-05-15': '134',
  '2013-06-12': '162',
  '2015-08-24': '965',
  '2017-03-31': '1550',
  '2016-11-19': '1418',
  '2015-05-17': '866',
  '2013-10-07': '279',
  '2014-01-24': '388',
  '2016-09-18': '1356',
  '2017-05-12': '1592',
  '2015-09-24': '996',
  '2016-06-03': '1249',
  '2014-06-

In [30]:
train.dtypes

id               int32
date            object
store_nbr        int16
item_nbr         int32
unit_sales     float16
onpromotion       bool
dtype: object

In [31]:
train['date'] = train['date'].map(replace['date'].get)

In [32]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,0,25,103665,7.0,False
1,1,0,25,105574,1.0,False
2,2,0,25,105575,2.0,False
3,3,0,25,108079,1.0,False
4,4,0,25,108701,1.0,False


In [33]:
train['date'] = train['date'].astype('int16')

In [34]:
train.to_csv('./data/train_processed.csv', index=False)

In [3]:
# fill empty values (maybe) - group by store_nbr, item_nbr, join date and unit_sales
types = {'id': 'int32',
         'date': 'int16',
         'item_nbr': 'int32',
         'store_nbr': 'int16',
         'unit_sales': 'float16',
         'onpromotion': bool,
    }
train = pd.read_csv('./data/train_processed.csv', dtype=types, nrows=100000)
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,0,25,103665,7.0,False
1,1,0,25,105574,1.0,False
2,2,0,25,105575,2.0,False
3,3,0,25,108079,1.0,False
4,4,0,25,108701,1.0,False


In [ ]:
test = pd.read_csv('./data/test.csv', usecols=['store_nbr', 'item_nbr'])
test = list(set([tuple(x) for x in test.values]))

In [6]:
import itertools
comb = list(itertools.product(*[test, list(train['date'].unique())]))
comb = [(t[0], t[1], m) for t, m in comb]

train2 = pd.DataFrame(comb, columns=['item_nbr', 'store_nbr', 'date'])
train2 = train2.sort_values(by=['date', 'item_nbr', 'store_nbr'])
train2 = train2.merge(train, on=['date', 'item_nbr', 'store_nbr'], how='left')
train2 = train2.fillna(0.0)
train = train2

In [7]:
train.head()

,item_nbr,store_nbr,date,id,unit_sales,onpromotion
0,1,96995,0,0.0,0.0,0.0
1,1,96995,1,0.0,0.0,0.0
2,1,96995,2,0.0,0.0,0.0
3,1,96995,3,0.0,0.0,0.0
4,1,99197,0,0.0,0.0,0.0
